<a href="https://colab.research.google.com/github/vid19/Transmission-mode-selection-in-V2V/blob/master/ML_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML approach - unsupervised leanrning 

In [9]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [2]:
df = pd.read_csv("hlo.csv")

In [3]:
X = df[["Rsrp", "SINR"]]

In [4]:
kmeans = KMeans(n_clusters=3, random_state=42).fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
centroids = kmeans.cluster_centers_
print("Centroids:")
print(centroids)

Centroids:
[[-86.18474557  36.49071269]
 [-70.58590319  52.65318644]
 [-78.61933742  44.61965656]]


In [6]:
df["Transmission_mode"] = kmeans.labels_


In [ ]:
df

,% time,cellId,IMSI,RNTI,rsrp,sinr,Rsrp,SINR,Transmission_mode
0,0.000214,1,8,0,2.250000e-11,47372.0,-76.48,93.510434,1
1,0.000214,1,9,0,3.810000e-11,80324.1,-74.19,98.096917,1
2,0.000214,1,11,0,8.900000e-12,18769.0,-80.50,85.468823,1
3,0.000214,1,15,0,3.820000e-11,80512.7,-74.18,98.117288,1
4,0.000214,1,16,0,2.950000e-11,62137.3,-75.31,95.867048,1
...,...,...,...,...,...,...,...,...,...
201061,2.872210,3,19,9,1.310000e-11,27551.7,-78.84,88.802968,1
201062,2.872210,3,24,2,3.760000e-11,79337.9,-74.24,97.989614,1
201063,2.872210,3,30,1,9.450000e-12,19932.4,-80.24,85.991192,1
201064,2.872210,3,48,3,2.750000e-11,57912.4,-75.61,95.255431,1


In [7]:
df["Transmission_mode"].replace({0: "Single-hop relay", 1: "Multi-hop relay", 2: "Client"}, inplace=True)

In [ ]:
df.to_csv("DlRsrpSinr_stats_with_transmission_mode.csv", index=False)

another approach - decision tree classifier

In [ ]:
!pip install --upgrade scikit-learn





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
data = pd.read_csv('hlo.csv')
print(data.columns)



Index(['% time', 'cellId', 'IMSI', 'RNTI', 'rsrp', 'sinr', 'Rsrp', 'SINR'], dtype='object')


In [ ]:
X = data[['SINR', 'Rsrp']]
y = []
for i in range(len(data)):
    if data['Rsrp'][i] >= -60 and data['Rsrp'][i] >= -70:
        y.append('Multi-hop')
    elif data['Rsrp'][i] >= -70 and data['Rsrp'][i] >= -75:
        y.append('Single-hop')
    else:
        y.append('Client')

data['TransmissionMode'] = y

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.feature_selection import SelectKBest, mutual_info_classif


selector = SelectKBest(mutual_info_classif, k=2)
selector.fit(X_train, y_train)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)


from sklearn.tree import DecisionTreeClassifier


dt = DecisionTreeClassifier(random_state=42)


dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:


y_pred = dt.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


#plot_tree(dt, feature_names=['Sinr', 'Rsrp'], class_names=['Client', 'Single-hop', 'Multi-hop'], filled=True)


#plot_confusion_matrix(dt, X_test, y_test, display_labels=['Client', 'Single-hop', 'Multi-hop'])

Accuracy: 1.0
              precision    recall  f1-score   support

      Client       1.00      1.00      1.00    196114
   Multi-hop       1.00      1.00      1.00      1154
  Single-hop       1.00      1.00      1.00     12447

    accuracy                           1.00    209715
   macro avg       1.00      1.00      1.00    209715
weighted avg       1.00      1.00      1.00    209715

